In [ ]:
!pip install -q langchain langchain-huggingface chromadb huggingface_hub sentence-transformers
!pip install -q langchain-community langchainhub pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 120.0 MB/s eta

### Import the necessary libraries

In [ ]:
import os
from huggingface_hub import login
from google.colab import files
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import warnings

warnings.filterwarnings("ignore")

In [ ]:
login() 

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1" 
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

text_gen_pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_new_tokens=512, 
    temperature=0.7, 
    pad_token_id=tokenizer.eos_token_id,
    do_sample=True 
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer",
    k=5
)

In [ ]:
def load_and_chunk_uploaded_pdf(chunk_size=500, chunk_overlap=100):
    uploaded = files.upload() 
    pdf_path = list(uploaded.keys())[0]
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

In [ ]:
def create_vectorstore(docs, embedding_model, persist_directory="interviewrag_chroma"):
    return Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_name="interviewrag-docs"
    )

In [ ]:
def upload_and_prepare_vectorstore(embedding_model):
    print("📂 Please upload your interview PDF...")
    chunks = load_and_chunk_uploaded_pdf()
    print(f"✅ Loaded and split {len(chunks)} chunks.")
    vectorstore = create_vectorstore(chunks, embedding_model)
    print("📦 Vectorstore created successfully.")
    return vectorstore

In [ ]:
def expand_query(llm, user_input):
    prompt = f"Rewrite this interview question to be more specific and detailed: {user_input}"
    return llm.invoke(prompt)

In [ ]:
QA_TEMPLATE = """
You are a helpful AI assistant tasked with answering interview-related questions using only the provided context.

Question:
{question}

Relevant Context:
{context}

Instructions:
- Base your answer only on the context above.
- Do not make up information beyond the context.
- If the answer is not available, say:
  "I'm sorry, the provided materials do not include an answer to this specific question."
- Provide a helpful, friendly, and well-structured answer that is informative but not too lengthy.
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(QA_TEMPLATE)

In [ ]:
def create_rag_chain(llm, retriever, memory):
    return ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=True,
        combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
        output_key="answer",
        verbose=True
    )

In [ ]:
def interview_rag_chat(llm, vectorstore, memory):

    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    rag_chain = create_rag_chain(llm, retriever, memory)

    while True:
        user_input = input("📝 Ask your interview question (or type 'exit'):\n> ")
        if user_input.lower().strip() == "exit":
            print("👋 Session ended.")
            break

        expanded = expand_query(llm, user_input)
        result = rag_chain.invoke(expanded)
        print("\n🤖 Answer:\n", result["answer"], "\n")

In [ ]:
vectorstore = upload_and_prepare_vectorstore(embedding_model)

📂 Please upload your interview PDF...


Saving interview_chat_summary.pdf to interview_chat_summary.pdf
✅ Loaded and split 6 chunks.
📦 Vectorstore created successfully.


In [ ]:
interview_rag_chat(llm, vectorstore, memory)

📝 Ask your interview question (or type 'exit'):
> > What should I do before going to an interview?


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful AI assistant tasked with answering interview-related questions using only the provided context.

Question:
Rewrite this interview question to be more specific and detailed: > What should I do before going to an interview?

1. Research the company: Look up the company's mission, values, culture, and recent news or developments. This will help you understand the company's goals and priorities and show that you're genuinely interested in the job.
2. Review the job description: Read through the job description again to make sure you understand the responsibilities and requirements for the position. This will help you prepare questions to ask the interviewer and show that you're serious about the job.
3. Choose an appropriate outfit: Dress professionally and appropriatel